In [4]:
import pandas as pd
import numpy as np
# Read the data from CSV files
dados = pd.read_csv('tripadvisor_hotel_reviews.csv', error_bad_lines=False)


In [60]:
dados_to = dados['Review'].tolist()

In [66]:
def cleanup(text):
    #import string
    punctuation = '[!-.:?;"@\n_/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

dados_cn = []
for i in dados_to:
    i = cleanup(i)
    dados_cn.append(i)

In [14]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
tuned_parameters = {
    'vect__ngram_range': [(1, 1), (1, 2), (2, 2)],
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': [1, 1e-1, 1e-2]}

In [70]:
x_train, x_test, y_train, y_test = train_test_split(dados_cn, dados['Rating'], test_size=0.3, random_state=42)

In [73]:
from sklearn.metrics import classification_report
clf = GridSearchCV(text_clf, tuned_parameters, cv=10)
clf.fit(x_train, y_train)

print(classification_report(y_test, clf.predict(x_test), digits=4))

              precision    recall  f1-score   support

           1     0.7545    0.3843    0.5092       432
           2     0.3647    0.2447    0.2929       523
           3     0.3012    0.0388    0.0687       645
           4     0.4370    0.3969    0.4160      1887
           5     0.6243    0.8869    0.7328      2661

    accuracy                         0.5576      6148
   macro avg     0.4964    0.3903    0.4039      6148
weighted avg     0.5200    0.5576    0.5128      6148

